In [1]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.prompts import PromptTemplate

try:
    storage_context = StorageContext.from_defaults(persist_dir='../storage/cache/andrew/medical')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('../assets/AndrewHuberman/medical').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/medical/')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [3]:
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "3.Câu trả lời nên thật chi tiết và dài"
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [4]:
from llama_index.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="sentence-transformers/all-mpnet-base-v2", top_n=3
)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[rerank],
    text_qa_template = text_qa_template
)

In [ ]:
now = time()
response = query_engine.query(
    "Bệnh bạch hầu là gì?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

In [ ]:
print(response)

In [ ]:
print(response.get_formatted_sources(length=1000))

In [ ]:
print(response.metadata)

In [ ]:
response.response

# 2. Đánh giá RAG

In [6]:
import pandas as pd

df = pd.read_csv('/Users/duongtrongchi/project/RAG/rag-app/test/test_case.csv')

df.head()

,Questions,References,Level
0,Ăn không tiêu là bệnh gì?,<em><strong>Ăn không tiêu</strong> là tình trạ...,1.0
1,Cách chữa bệnh ăn không tiêu?,<h2><strong>Cách chữa ăn không tiêu</strong></...,NaN
2,Bệnh cúm là bệnh gì?,<h2><strong>Bệnh cúm là gì?</strong></h2> <str...,NaN
3,Bệnh cúm có các triệu chứng gì?,<h2><strong>Triệu chứng cúm</strong></h2> Cúm ...,NaN
4,Trĩ ngoại có bao nhiêu cấp độ?,</h3> Trĩ ngoại là tình trạng búi trĩ phát tri...,NaN


In [12]:
from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index.embeddings import SimilarityMode


evaluator = SemanticSimilarityEvaluator(
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)

list_res = []
list_result = []
passed = []
for i in range(0, len(df)):
    reference = df['References'][i]
    response = query_engine.query(df['Questions'][i])

    result = await evaluator.aevaluate(
        response=response.response,
        reference=reference,
    )

    list_res.append(response.response)
    list_result.append(result.score)
    passed.append(result.passing)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [9]:
df['Result'] = list_result
df['Passed'] = passed
df['Responses'] = list_res

In [11]:
df.to_csv('../results.csv', index=False)

In [ ]:
# Embedding Similarity Evaluator

# from llama_index.evaluation import SemanticSimilarityEvaluator
# from llama_index.embeddings import SimilarityMode


# evaluator = SemanticSimilarityEvaluator(
#     similarity_mode=SimilarityMode.DEFAULT,
#     similarity_threshold=0.9,
# )

# reference = """
#  <b>Bệnh bạch hầu là gì?</b> Bệnh bạch hầu là bệnh nhiễm khuẩn cấp tính có giả mạc ở tuyến hạnh nhân, hầu họng, thanh quản, mũi. Bệnh có thể xuất hiện ở da, các màng niêm mạc khác như kết mạc mắt hoặc bộ phận sinh dục. Đây là một bệnh vừa nhiễm trùng vừa nhiễm độc và các tổn thương nghiêm trọng của bệnh chủ yếu là do ngoại độc tố của vi khuẩn bạch hầu gây ra.
# """

# response = query_engine.query(
#     "Bệnh bạch hầu là gì?",
# )

# result = await evaluator.aevaluate(
#     response=response.response,
#     reference=reference,
# )

# print("Score: ", result.score)
# print("Passing: ", result.passing)